# MONALISA T4.1 - EO indices; Sentinel-2

This notebook allows computing 28 RS indices from Sentinel-2 based on the "Indicators_vs_Pressures.xslx" document; enhacing reproducibility and comparability

* **Sentinel-2 True Color**
* **Soil indices:** Normalized Difference Soil Index (NDSI); Bare Soil Index (BSI); modified Bare Soil Index (MBI)
* **Soil Salinity indices:**  Soil Salinity Index-1 (SSI1); Soil Salinity Index-2 (SSI-2); Soil Salinity Index-3 (SSI-3); Salinitiy Index (SI); Soil Adjusted Salinity Index (SASI); Normalized Difference Salinity Index (NDSaI)
* **Vegetation indices:** Normalized Difference Vegetation Index (NDVI); Red Edge Position (REP); Green Normalized Difference Vegetation Index (GNDVI); Soil Adjusted Vegetation Index (SAVI); Modified Soil Adjusted Vegetation Index (MSAVI); Modified Soil Adjusted Vegetation Index-2 (MSAVI2); Optimized Soil-Adjusted Vegetation Index (OSAVI), Normalized Difference Red Edge (NDRE); Atmospherically Resistant Vegetation Index (ARVI); Enhanced Vegeation Index (EVI); Enhanced Vegetation Index-2 (EVI2); Fractional Vegetation Cover (FVC); Color Infrared (CIR); Plant Senescence Reflectance Index (PSRI); Normalized Difference Chlorophyll Index (NDCI); Normalized Difference Tillage Index (NDTI); Green Red Vegetation Index (GRVI); Visible Atmospherically Resistent Index (VARI)
* **Burned Area indices:** Normalized Burn Ratio-1 (NBR1); Normalized Burn Ratio-2 (NBR2)
* **Drought indices:** Normalized Difference Drought Index (NDDI); Desertification Soil Index (DSI); Moisture Stress Index (MSI); Normalized Multiband Drought Index (NMDI); Visible-SWIR Drought Index (VSDI); Redness Index (RDI); Desertification index (DI)
* **Water indices:** Normalized Difference Water Index-1 (NDWI1); Normalized Difference Water Index-2 (NDWI2); modified Normalized Difference Water Index (MNDWI)

Author(s): Gregory Giuliani, Audrey Lambiel [University of Geneva], Ioannis Manakos [CERTH], Imma Serra [CREAF]

Version: 1.2 [10.11.2025]


## Initialization

In [14]:
import ee
import geemap

In [15]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [16]:
#Config
startDate = '2024-01-01'
endDate = '2025-01-01'
site_name = 'LosPedroches'
year = '2024'

In [ ]:
#AoI using assets
aoi = ee.FeatureCollection('projects/ee-rs2/assets/LosPedroches') #alta_murgia; asterousia; medenine; tayasir; LosPedroches; BerchiddaMonti
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 14)

#### *Sentinel-2 - True color*

In [18]:
#Sentinel-2
#Cloud masking 
def cloudMask(image):
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(10)))
    return image.updateMask(mask.eq(0))

#Import Sentinel-2 image collection
CollectionS2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
              .filterBounds(aoi) \
              .filterDate(startDate, endDate) \
              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
              .map(cloudMask)

#RGB visualization parameters
visualizationS2 = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 1800,
}

#Visualize RGB median
MedianS2 = CollectionS2.median().clip(aoi)
Map.addLayer(MedianS2, visualizationS2, 'Sentinel-2 | True Color')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=MedianS2, 
  description='RGB_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='RGB_S2_'+site_name+'_'+year,
  folder=site_name,
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Soil Indices

#### *Sentinel-2 - NDSI*

In [ ]:
#Normalized Difference Soil Index
ndsi = MedianS2.normalizedDifference(['B3', 'B11']).rename('NDSI')

# Compute min and max values over the AOI
ndsi_stats = ndsi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndsi_stats.get('NDSI_min').getInfo(),
    'max': ndsi_stats.get('NDSI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndsi, vis_params, 'Sentinel-2 | NDSI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndsi, 
  description='NDSI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDSI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - BSI*

In [9]:
#Bare Soil Index
bsi = MedianS2.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1+RED) + (NIR +BLUE))', {
            'SWIR1': MedianS2.select('B11'),
            'RED': MedianS2.select('B4'),
            'NIR': MedianS2.select('B8'),
            'BLUE': MedianS2.select('B2')
        }).rename('BSI')

# Compute min and max values over the AOI
bsi_stats = bsi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': bsi_stats.get('BSI_min').getInfo(),
    'max': bsi_stats.get('BSI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(bsi, vis_params, 'Sentinel-2 | BSI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=bsi, 
  description='BSI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='BSI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - MBI*

In [12]:
#modified Bare Soil Index
mbi = MedianS2.expression(
        '((SWIR1 - SWIR2 - NIR) / (SWIR1 + SWIR2 + NIR)) + 0.5', {
            'SWIR1': MedianS2.select('B11'),
            'SWIR2': MedianS2.select('B12'),
            'NIR': MedianS2.select('B8')
        }).rename('MBI')

# Compute min and max values over the AOI
mbi_stats = mbi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': mbi_stats.get('MBI_min').getInfo(),
    'max': mbi_stats.get('MBI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(mbi, vis_params, 'Sentinel-2 | MBI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=mbi, 
  description='MBI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MBI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Soil Salinity Indices

#### *Sentinel-2 - SSI1*

In [19]:
#Soil Salinity Index-1
ssi1 = MedianS2.expression(
        'B2/B4', {
            'B2': MedianS2.select('B2'),
            'B4': MedianS2.select('B4'),
        }).rename('SSI1')

# Compute min and max values over the AOI
ssi1_stats = ssi1.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ssi1_stats.get('SSI1_min').getInfo(),
    'max': ssi1_stats.get('SSI1_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ssi1, vis_params, 'Sentinel-2 | SSI1')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ssi1, 
  description='SSI1_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SSI1_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - SSI2*

In [16]:
#Soil Salinity Index-2
ssi2 = MedianS2.normalizedDifference(['B2', 'B4']).rename('SSI2')

# Compute min and max values over the AOI
ssi2_stats = ssi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ssi2_stats.get('SSI2_min').getInfo(),
    'max': ssi2_stats.get('SSI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ssi2, vis_params, 'Sentinel-2 | SSI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ssi2, 
  description='SSI2_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SSI2_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - SSI3*

In [18]:
#Soil Salinity Index-3
ssi3 = MedianS2.expression(
        '(B3*B4)/B2', {
            'B2': MedianS2.select('B2'),
            'B3': MedianS2.select('B3'),
            'B4': MedianS2.select('B4')
        }).rename('SSI3')

# Compute min and max values over the AOI
ssi3_stats = ssi3.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ssi3_stats.get('SSI3_min').getInfo(),
    'max': ssi3_stats.get('SSI3_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ssi3, vis_params, 'Sentinel-2 | SSI3')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ssi3, 
  description='SSI3_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SSI3_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - SI*

In [22]:
#Salinitiy Index
si = MedianS2.expression(
        'sqrt(B3*B2)', {
            'B2': MedianS2.select('B2'),
            'B3': MedianS2.select('B3')
        }).rename('SI')

# Compute min and max values over the AOI
si_stats = si.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': si_stats.get('SI_min').getInfo(),
    'max': si_stats.get('SI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(si, vis_params, 'Sentinel-2 | SI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=si, 
  description='SI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - SASI*

In [7]:
#Soil Adjusted Salinity Index
sasi = MedianS2.expression(
        '(B11-B8)/(B11+B8+0.5)', {
            'B11': MedianS2.select('B11'),
            'B8': MedianS2.select('B8')
        }).rename('SASI')

# Compute min and max values over the AOI
sasi_stats = sasi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': sasi_stats.get('SASI_min').getInfo(),
    'max': sasi_stats.get('SASI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(sasi, vis_params, 'Sentinel-2 | SASI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=sasi, 
  description='SASI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SASI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NDSaI*

In [ ]:
#Normalized Difference Salinity Index
ndsai = MedianS2.normalizedDifference(['B11', 'B8']).rename('NDSaI')

# Compute min and max values over the AOI
ndsai_stats = ndsai.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndsai_stats.get('NDSaI_min').getInfo(),
    'max': ndsai_stats.get('NDSaI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndsai, vis_params, 'Sentinel-2 | NDSaI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndsai, 
  description='NDSaI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDSaI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Vegetation Indices

#### *Sentinel-2 - NDVI*

In [30]:
#Normalized Difference Vegetation Index
ndvi = MedianS2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Compute min and max values over the AOI
ndvi_stats = ndvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndvi_stats.get('NDVI_min').getInfo(),
    'max': ndvi_stats.get('NDVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndvi, vis_params, 'Sentinel-2 | NDVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndvi, 
  description='NDVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - REP*

In [32]:
#Red Edge Position
rep = MedianS2.expression(
        '700+40*((((B4+B7)/2)-B5)/(B6-B5))', {
            'B4': MedianS2.select('B4'),
            'B5': MedianS2.select('B5'),
            'B6': MedianS2.select('B6'),
            'B7': MedianS2.select('B7')
        }).rename('REP')

# Compute min and max values over the AOI
rep_stats = rep.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': rep_stats.get('REP_min').getInfo(),
    'max': rep_stats.get('REP_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(rep, vis_params, 'Sentinel-2 | REP')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=rep, 
  description='REP_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='REP_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - GNDVI*

In [34]:
#Green Normalized Difference Vegetation Index
gndvi = MedianS2.normalizedDifference(['B8', 'B3']).rename('GNDVI')

# Compute min and max values over the AOI
gndvi_stats = gndvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': gndvi_stats.get('GNDVI_min').getInfo(),
    'max': gndvi_stats.get('GNDVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(gndvi, vis_params, 'Sentinel-2 | GNDVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=gndvi, 
  description='GNDVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='GNDVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - SAVI*

In [6]:
#Soil Adjusted Vegetation Index
savi = MedianS2.expression(
        '1.5*((NIR-RED)/(NIR+RED+0.5))', {
            'NIR': MedianS2.select('B8'),
            'RED': MedianS2.select('B4')
        }).rename('SAVI')

# Compute min and max values over the AOI
savi_stats = savi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': savi_stats.get('SAVI_min').getInfo(),
    'max': savi_stats.get('SAVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(savi, vis_params, 'Sentinel-2 | SAVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=savi, 
  description='SAVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='SAVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - MSAVI*

In [36]:
#Modified Soil Adjusted Vegetation Index
msavi = MedianS2.expression(
        '((2 * NIR + 1 - sqrt((2 * NIR + 1)**2- 8 * (NIR - RED)))/2)', {
            'NIR': MedianS2.select('B8'),
            'RED': MedianS2.select('B4')
        }).rename('MSAVI')

# Compute min and max values over the AOI
msavi_stats = msavi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': msavi_stats.get('MSAVI_min').getInfo(),
    'max': msavi_stats.get('MSAVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(msavi, vis_params, 'Sentinel-2 | MSAVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=msavi, 
  description='MSAVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MSAVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - MSAVI2*

In [38]:
#Modified Soil Adjusted Vegetation Index 2
msavi2 = MedianS2.expression(
        '0.5*((2 * NIR + 1 - sqrt((2 * NIR + 1)**2- 8 * (NIR - RED)))/2)', {
            'NIR': MedianS2.select('B8'),
            'RED': MedianS2.select('B4')
        }).rename('MSAVI2')

# Compute min and max values over the AOI
msavi2_stats = msavi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': msavi2_stats.get('MSAVI2_min').getInfo(),
    'max': msavi2_stats.get('MSAVI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(msavi2, vis_params, 'Sentinel-2 | MSAVI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=msavi2, 
  description='MSAVI2_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MSAVI2_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - OSAVI*

In [40]:
#Optimized Soil-Adjusted Vegetation Index
osavi = MedianS2.expression(
        '(B8-B4)/(B8+B4+0.5)', {
            'B8': MedianS2.select('B8'),
            'B4': MedianS2.select('B4'),
        }).rename('OSAVI')

# Compute min and max values over the AOI
osavi_stats = osavi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': osavi_stats.get('OSAVI_min').getInfo(),
    'max': osavi_stats.get('OSAVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(osavi, vis_params, 'Sentinel-2 | OSAVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=osavi, 
  description='OSAVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='OSAVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NDRE*

In [42]:
#Normalized Difference Red Edge
ndre = MedianS2.normalizedDifference(['B8', 'B5']).rename('NDRE')


# Compute min and max values over the AOI
ndre_stats = ndre.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndre_stats.get('NDRE_min').getInfo(),
    'max': ndre_stats.get('NDRE_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndre, vis_params, 'Sentinel-2 | NDRE')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndre, 
  description='NDRE_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDRE_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - ARVI*

In [45]:
#Atomspherically Resistant Vegetation Index
arvi = MedianS2.expression(
        '(B8A - B4 - (0.106*(B4-B2)))/(B8A + B4 - (0.106*(B4-B2)))', {
            'B8A': MedianS2.select('B8A'),
            'B4': MedianS2.select('B4'),
            'B2': MedianS2.select('B2')
        }).rename('ARVI')

# Compute min and max values over the AOI
arvi_stats = arvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': arvi_stats.get('ARVI_min').getInfo(),
    'max': arvi_stats.get('ARVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(arvi, vis_params, 'Sentinel-2 | ARVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=arvi, 
  description='ARVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='ARVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - EVI*

In [47]:
#Enhanced Vegetation Index (EVI)
evi = MedianS2.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': MedianS2.select('B8'),
            'RED': MedianS2.select('B4'),
            'BLUE': MedianS2.select('B2')
        }).rename('EVI')

# Compute min and max values over the AOI
evi_stats = evi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': evi_stats.get('EVI_min').getInfo(),
    'max': evi_stats.get('EVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(evi, vis_params, 'Sentinel-2 | EVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=evi, 
  description='EVI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='EVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - EVI2*

In [49]:
#Enhanced Vegetation Index 2 (EVI2)
evi2 = MedianS2.expression(
        '2.5 * ((NIR - RED) / (NIR + RED + 1))', {
            'NIR': MedianS2.select('B8'),
            'RED': MedianS2.select('B4')
        }).rename('EVI2')

# Compute min and max values over the AOI
evi2_stats = evi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': evi2_stats.get('EVI2_min').getInfo(),
    'max': evi2_stats.get('EVI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize EVI2
Map.addLayer(evi2, vis_params, 'Sentinel-2 | EVI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=evi2, 
  description='EVI2_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='EVI2_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - FVC*

In [51]:
#Fractional Vegetation Cover (FVC)
ndviStats = ndvi.reduceRegion(**{
  'reducer': ee.Reducer.minMax(),
  'geometry': aoi,
  'scale': 10,
  'maxPixels': 1e9
})
ndviMin = ee.Number(ndviStats.get('NDVI_min'))
ndviMax = ee.Number(ndviStats.get('NDVI_max'))

# Visualization parameters
vis_params = {
    'min': ndviMin,
    'max': ndviMax,
    'palette': ['red', 'white', 'green']
}

#Fractional Vegetation Cover
fvc = ((ndvi.subtract(ndviMin)).divide(ndviMax.subtract(ndviMin))) \
          .pow(ee.Number(2)) \
          .rename('FVC')

Map.addLayer(fvc, vis_params, 'Sentinel-2 | FVC')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=fvc, 
  description='FVC_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='FVC_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - CIR*

In [20]:
#Color Infrared
cir = MedianS2.expression(
        '(B8/B5)-1', {
            'B8': MedianS2.select('B8'),
            'B5': MedianS2.select('B5')
        }).rename('CIR')

# Compute min and max values over the AOI
cir_stats = cir.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': cir_stats.get('CIR_min').getInfo(),
    'max': cir_stats.get('CIR_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(cir, vis_params, 'Sentinel-2 | CIR')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=cir, 
  description='CIR_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='CIR_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - PSRI*

In [28]:
#Plant Senescence Reflectance Index
psri = MedianS2.expression(
        '(B4-B2)/B6', {
            'B2': MedianS2.select('B2'),
            'B4': MedianS2.select('B4'),
            'B6': MedianS2.select('B6')
        }).rename('PSRI')

# Compute min and max values over the AOI
psri_stats = psri.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': psri_stats.get('PSRI_min').getInfo(),
    'max': psri_stats.get('PSRI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(psri, vis_params, 'Sentinel-2 | PSRI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=psri, 
  description='PSRI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='PSRI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NDCI*

In [ ]:
#Normalized Difference Chlorophyll Index
ndci = MedianS2.normalizedDifference(['B3', 'B2']).rename('NDCI')


# Compute min and max values over the AOI
ndci_stats = ndci.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndci_stats.get('NDCI_min').getInfo(),
    'max': ndci_stats.get('NDCI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndci, vis_params, 'Sentinel-2 | NDCI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndci, 
  description='NDCI_S2__'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDCI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NDTI*

In [22]:
#Normalized Difference Tillage Index
ndti = MedianS2.normalizedDifference(['B11', 'B12']).rename('NDTI')


# Compute min and max values over the AOI
ndti_stats = ndti.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndti_stats.get('NDTI_min').getInfo(),
    'max': ndti_stats.get('NDTI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndti, vis_params, 'Sentinel-2 | NDTI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndti, 
  description='NDTI_S2__'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDTI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - GRVI*

In [23]:
#Green Red Vegetation Index
grvi = MedianS2.normalizedDifference(['B3', 'B4']).rename('GRVI')


# Compute min and max values over the AOI
grvi_stats = grvi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': grvi_stats.get('GRVI_min').getInfo(),
    'max': grvi_stats.get('GRVI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(grvi, vis_params, 'Sentinel-2 | GRVI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=grvi, 
  description='GRVI_S2__'+site_name+'_'+year+'_Layer',
  fileNamePrefix='GRVI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - RGRI*

In [ ]:
#Red Green Ratio Index
rgri = MedianS2.expression(
        'B4/B3', {
            'B4': MedianS2.select('B4'),
            'B3': MedianS2.select('B3')
        }).rename('RGRI')

# Compute min and max values over the AOI
rgri_stats = rgri.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': rgri_stats.get('RGRI_min').getInfo(),
    'max': rgri_stats.get('RGRI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(rgri, vis_params, 'Sentinel-2 | RGRI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=rgri, 
  description='RGRI_S2__'+site_name+'_'+year+'_Layer',
  fileNamePrefix='RGRI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - VARI*

In [26]:
#Visible Atmospherically Resistent Index
vari = MedianS2.expression(
        '(B3-B4)/(B3+B4-B2)', {
            'B4': MedianS2.select('B4'),
            'B3': MedianS2.select('B3'),
            'B2': MedianS2.select('B2')
        }).rename('VARI')

# Compute min and max values over the AOI
vari_stats = vari.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': vari_stats.get('VARI_min').getInfo(),
    'max': vari_stats.get('VARI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(vari, vis_params, 'Sentinel-2 | VARI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=vari, 
  description='VARI_S2__'+site_name+'_'+year+'_Layer',
  fileNamePrefix='VARI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Burned Areas Indices

#### *Sentinel-2 - NBR*

In [53]:
#Normalized Burn Ratio-1
nbr1 = MedianS2.normalizedDifference(['B8', 'B12']).rename('NBR1')

# Compute min and max values over the AOI
nbr1_stats = nbr1.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nbr1_stats.get('NBR1_min').getInfo(),
    'max': nbr1_stats.get('NBR1_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize NBR
Map.addLayer(nbr1, vis_params, 'Sentinel-2 | NBR1')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nbr1, 
  description='NBR1_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NBR1_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NBR2*

In [55]:
#Normalized Burn Ratio-2
nbr2 = MedianS2.normalizedDifference(['B11', 'B12']).rename('NBR2')

# Compute min and max values over the AOI
nbr2_stats = nbr2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nbr2_stats.get('NBR2_min').getInfo(),
    'max': nbr2_stats.get('NBR2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(nbr2, vis_params, 'Sentinel-2 | NBR2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nbr2, 
  description='NBR2_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NBR2_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Drought Indices

#### *Sentinel-2 - NDDI*

In [9]:
#Normalized Difference Drought Index
nddi = MedianS2.expression(
        '(((B8-B4)/(B8+B4)) - ((B3-B8)/(B3+B8)))/(((B8-B4)/(B8+B4)) + ((B3-B8)/(B3+B8)))', {
            'B8': MedianS2.select('B8'),
            'B4': MedianS2.select('B4'),
            'B3': MedianS2.select('B3')
        }).rename('NDDI')

# Compute min and max values over the AOI
nddi_stats = nddi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nddi_stats.get('NDDI_min').getInfo(),
    'max': nddi_stats.get('NDDI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(nddi, vis_params, 'Sentinel-2 | NDDI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nddi, 
  description='NDDI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDDI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - DSI*

In [58]:
#Desertification Soil Index
dsi = MedianS2.expression(
        '((B11+B4)-(B8+B2))/((B11+B4)+(B8+B2))', {
            'B2': MedianS2.select('B2'),
            'B4': MedianS2.select('B4'),
            'B8': MedianS2.select('B8'),
            'B11': MedianS2.select('B11')
        }).rename('DSI')

# Compute min and max values over the AOI
dsi_stats = dsi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': dsi_stats.get('DSI_min').getInfo(),
    'max': dsi_stats.get('DSI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(dsi, vis_params, 'Sentinel-2 | DSI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=dsi, 
  description='DSI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='DSI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - MSI*

In [29]:
#Moisture Soil Index
msi = MedianS2.expression(
        'B11/B8', {
            'B11': MedianS2.select('B11'),
            'B8': MedianS2.select('B8')
        }).rename('MSI')

# Compute min and max values over the AOI
msi_stats = msi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': msi_stats.get('MSI_min').getInfo(),
    'max': msi_stats.get('MSI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(msi, vis_params, 'Sentinel-2 | MSI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=msi, 
  description='MSI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MSI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NMDI*

In [30]:
#Normalized Multiband Drought Index
nmdi = MedianS2.expression(
        '(B8-(B11-B12))/(B8+(B11-B12))', {
            'B11': MedianS2.select('B11'),
            'B8': MedianS2.select('B8'),
            'B12': MedianS2.select('B12')
        }).rename('NMDI')

# Compute min and max values over the AOI
nmdi_stats = nmdi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': nmdi_stats.get('NMDI_min').getInfo(),
    'max': nmdi_stats.get('NMDI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(nmdi, vis_params, 'Sentinel-2 | NMDI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=nmdi, 
  description='NMDI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NMDI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - OGSI*
OGSI (scene-wise construction)
        Robust 5–95% rescale to [0,1]: NDVÎ, BSÎ, MSÎ
        NDVI_dry = 1 − NDVÎ
OGSI = mean(NDVI_dry, BSÎ, MSÎ)

#### *Sentinel-2 - VSDI*

In [12]:
#Visible–SWIR Drought Index
ndvi = MedianS2.normalizedDifference(['B8', 'B4'])
bsi = MedianS2.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1+RED) + (NIR +BLUE))', {
            'SWIR1': MedianS2.select('B11'),
            'RED': MedianS2.select('B4'),
            'NIR': MedianS2.select('B8'),
            'BLUE': MedianS2.select('B2')
        })


vsdi = MedianS2.expression(
    '(bsi-ndvi)/(bsi+ndvi)',
    {
        'ndvi': ndvi,
        'bsi': bsi
    }
).rename('VSDI')

# Compute min and max values over the AOI
vsdi_stats = vsdi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': vsdi_stats.get('VSDI_min').getInfo(),
    'max': vsdi_stats.get('VSDI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(vsdi, vis_params, 'Sentinel-2 | VSDI')

#### *Sentinel-2 - RDI*

In [10]:
#Redness Index
ndvi = MedianS2.normalizedDifference(['B8', 'B4'])
bsi = MedianS2.expression(
        '((SWIR1 + RED) - (NIR + BLUE)) / ((SWIR1+RED) + (NIR +BLUE))', {
            'SWIR1': MedianS2.select('B11'),
            'RED': MedianS2.select('B4'),
            'NIR': MedianS2.select('B8'),
            'BLUE': MedianS2.select('B2')
        })


rdi = MedianS2.expression(
    '(1 - ndvi) * (1 + bsi)',
    {
        'ndvi': ndvi,
        'bsi': bsi
    }
).rename('RDI')

# Compute min and max values over the AOI
rdi_stats = rdi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': rdi_stats.get('RDI_min').getInfo(),
    'max': rdi_stats.get('RDI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(rdi, vis_params, 'Sentinel-2 | RDI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=rdi, 
  description='RDI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='RDI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - DI*

In [13]:
#Desertification Index
ndvi = MedianS2.normalizedDifference(['B8', 'B4'])
msi = MedianS2.expression(
        'B11/B8', {
            'B11': MedianS2.select('B11'),
            'B8': MedianS2.select('B8')
        })


di = MedianS2.expression(
    '(msi-ndvi)/(msi+ndvi)',
    {
        'ndvi': ndvi,
        'msi': msi
    }
).rename('DI')

# Compute min and max values over the AOI
di_stats = di.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': di_stats.get('DI_min').getInfo(),
    'max': di_stats.get('DI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(di, vis_params, 'Sentinel-2 | DI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=di, 
  description='DI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='DI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

## Water Indices

#### *Sentinel-2 - NDWI1*

In [60]:
#Normalized Difference Water Index-1
ndwi1 = MedianS2.normalizedDifference(['B8', 'B11']).rename('NDWI1')

# Compute min and max values over the AOI
ndwi1_stats = ndwi1.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndwi1_stats.get('NDWI1_min').getInfo(),
    'max': ndwi1_stats.get('NDWI1_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndwi1, vis_params, 'Sentinel-2 | NDWI1')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndwi1, 
  description='NDWI1_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDWI1_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - NDWI2*

In [62]:
#Normalized Difference Water Index-2
ndwi2 = MedianS2.normalizedDifference(['B3', 'B8']).rename('NDWI2')

# Compute min and max values over the AOI
ndwi2_stats = ndwi2.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': ndwi2_stats.get('NDWI2_min').getInfo(),
    'max': ndwi2_stats.get('NDWI2_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(ndwi2, vis_params, 'Sentinel-2 | NDWI2')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndwi2, 
  description='NDWI2_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='NDWI2_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)

#### *Sentinel-2 - MNDWI*

In [64]:
#modified Normalized Difference Water Index
mndwi = MedianS2.normalizedDifference(['B3', 'B11']).rename('MNDWI')

# Compute min and max values over the AOI
mndwi_stats = mndwi.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

# Visualization parameters
vis_params = {
    'min': mndwi_stats.get('MNDWI_min').getInfo(),
    'max': mndwi_stats.get('MNDWI_max').getInfo(),
    'palette': ['red', 'white', 'green']
}

#Visualize
Map.addLayer(mndwi, vis_params, 'Sentinel-2 | MNDWI')

In [ ]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=mndwi, 
  description='MNDWI_S2_'+site_name+'_'+year+'_Layer',
  fileNamePrefix='MNDWI_S2_'+site_name+'_'+year,
  folder=site_name,  
  region=aoi.geometry(),
  scale=10,
  crs='EPSG:4326',
  maxPixels=1e10
)